
<font size="20"><h3 align='center'>**_DSL Project_**</font><br>
    
    
<font size="5"><h3 align='center'><span style='color:Blue'> Predict impact of air quality on mortality rates(Kaggle Competition)</font>


<font size="3"><h3 align='center'>https://www.kaggle.com/c/predict-impact-of-air-quality-on-death-rates/overview</font><br>
    <br>


<font size="3">**Group Number - 2**<br>
<font size="3"> Mimansi Agarwal   &emsp;19BM6JP15</font>        
<font size="3"> Vaibhav HK Anand  &emsp;19BM6JP24</font>    
<font size="3"> Bankar Prasad Vilas &emsp;19BM6JP25</font> 


# Library

In [1]:

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from impyute.imputation.cs import mice
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import  make_scorer
import math

# Data import

In [2]:
train = pd.read_csv('C:/Users/deciPh3r/Desktop/Project/files/train.csv', parse_dates=['date'], index_col='Id')
test = pd.read_csv('C:/Users/deciPh3r/Desktop/Project/files/test.csv', parse_dates=['date'], index_col='Id')
region=pd.read_csv('C:/Users/deciPh3r/Desktop/Project/files/regions.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18403 entries, 1 to 18403
Data columns (total 8 columns):
region            18403 non-null object
date              18403 non-null datetime64[ns]
mortality_rate    18403 non-null float64
O3                18394 non-null float64
PM10              18394 non-null float64
PM25              15127 non-null float64
NO2               11833 non-null float64
T2M               18403 non-null float64
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 1.3+ MB


**_Missing values present in 4 columns, O3 ,PM10, PM25, NO2_**

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7886 entries, 18404 to 26289
Data columns (total 7 columns):
region    7886 non-null object
date      7886 non-null datetime64[ns]
O3        7886 non-null float64
PM10      7886 non-null float64
PM25      7886 non-null float64
NO2       7886 non-null float64
T2M       7886 non-null float64
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 492.9+ KB


In [5]:
region

,Code,Region
0,E12000001,North East
1,E12000002,North West
2,E12000003,Yorkshire and The Humber
3,E12000004,East Midlands
4,E12000005,West Midlands
5,E12000006,East of England
6,E12000007,London
7,E12000008,South East
8,E12000009,South West


In [6]:
region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
Code      9 non-null object
Region    9 non-null object
dtypes: object(2)
memory usage: 272.0+ bytes


# Exploratory data analysis

In [7]:
region.head()

,Code,Region
0,E12000001,North East
1,E12000002,North West
2,E12000003,Yorkshire and The Humber
3,E12000004,East Midlands
4,E12000005,West Midlands


#  Imputation

## Impution on full data

#  Feature Engeering

one hot encoding for regions

In [8]:
dummy1= pd.get_dummies(train['region'])
train_f = train.join(dummy1)

dummy2= pd.get_dummies(test['region'])
test_f = test.join(dummy2)

In [9]:
train_f.shape,test_f.shape

((18403, 17), (7886, 16))

doy and year

In [10]:
train_f['year'] = train_f['date'].map(lambda x: x.year)
test_f['year'] = test_f['date'].map(lambda x: x.year)

train_f['year^2'] = train_f['year']**2
test_f['year^2'] = test_f['year']**2

train_f['doy'] = train_f['date'].map(lambda x: x.dayofyear)
test_f['doy'] = test_f['date'].map(lambda x: x.dayofyear)

train_f['doy^2'] = train_f['doy']**2
test_f['doy^2'] = test_f['doy']**2

Log Transformation

In [11]:
train_f['NO2_log']=np.log(train_f['NO2'])
train_f['PM10_log']=np.log(train_f['PM10'])
train_f['PM25_log']=np.log(train_f['PM25'])

test_f['NO2_log']=np.log(test_f['NO2'])
test_f['PM10_log']=np.log(test_f['PM10'])
test_f['PM25_log']=np.log(test_f['PM25'])

In [12]:
train_f.info(),test_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18403 entries, 1 to 18403
Data columns (total 24 columns):
region            18403 non-null object
date              18403 non-null datetime64[ns]
mortality_rate    18403 non-null float64
O3                18394 non-null float64
PM10              18394 non-null float64
PM25              15127 non-null float64
NO2               11833 non-null float64
T2M               18403 non-null float64
E12000001         18403 non-null uint8
E12000002         18403 non-null uint8
E12000003         18403 non-null uint8
E12000004         18403 non-null uint8
E12000005         18403 non-null uint8
E12000006         18403 non-null uint8
E12000007         18403 non-null uint8
E12000008         18403 non-null uint8
E12000009         18403 non-null uint8
year              18403 non-null int64
year^2            18403 non-null int64
doy               18403 non-null int64
doy^2             18403 non-null int64
NO2_log           11833 non-null float64
PM10_log 

(None, None)

In [13]:
total_f=total=pd.concat([train_f,test_f],sort=False)

In [14]:
total_f['T2M_rolling'] = total_f.groupby('region')['T2M'].transform(lambda x: x.rolling(30).mean())

In [15]:
total_f.shape

(26289, 25)

In [16]:
train_f=total_f[:18403].dropna()
test_f=total_f[18403:].drop(['mortality_rate'],axis=1)

In [17]:
train_f.shape,test_f.shape

((11833, 25), (7886, 24))

#  Modelling - Regression based models

In [18]:
X_train, X_test, y_train, y_test = train_test_split(train_f.drop(['mortality_rate'],axis=1), train_f['mortality_rate'], test_size=0.2)

In [19]:
X_train.columns

Index(['region', 'date', 'O3', 'PM10', 'PM25', 'NO2', 'T2M', 'E12000001',
       'E12000002', 'E12000003', 'E12000004', 'E12000005', 'E12000006',
       'E12000007', 'E12000008', 'E12000009', 'year', 'year^2', 'doy', 'doy^2',
       'NO2_log', 'PM10_log', 'PM25_log', 'T2M_rolling'],
      dtype='object')

In [20]:
RMSE= make_scorer(metrics.mean_squared_error)

In [21]:
predictors=['O3','PM10','PM25','NO2','T2M']

In [22]:
X=train_f.drop(['mortality_rate'],axis=1)
y=train_f['mortality_rate']

In [23]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.25590564 0.2384088  0.23534413 0.24975171 0.21540355] 

average RMSE : 0.2389627659994164 
 

Lasso RMSE [0.26501773 0.24556641 0.2426855  0.25132788 0.21040855] 

average RMSE : 0.243001215350219 
 

Ridge RMSE [0.25590577 0.23840889 0.23534414 0.24975171 0.21540335] 

average RMSE : 0.2389627739056404 
 

RandomForestRegressor RMSE [0.27209648 0.25170445 0.26211702 0.26120663 0.21941687] 

average RMSE : 0.2533082876488943 
 



In [24]:
predictors=['O3','PM10','PM25','NO2_log','T2M']

In [25]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.26527521 0.24409334 0.24623856 0.25499492 0.21608415] 

average RMSE : 0.2453372337810475 
 

Lasso RMSE [0.27885318 0.25456861 0.25885199 0.26576436 0.21011588] 

average RMSE : 0.25363080541258476 
 

Ridge RMSE [0.26527777 0.24409446 0.24624209 0.25499278 0.21607701] 

average RMSE : 0.24533682390299894 
 

RandomForestRegressor RMSE [0.27209715 0.25170576 0.26211702 0.26120663 0.21941673] 

average RMSE : 0.253308657930002 
 



In [26]:
predictors=['O3','PM10','PM25_log','NO2_log','T2M']

In [27]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.26573681 0.24385109 0.24598684 0.25584064 0.21617526] 

average RMSE : 0.24551812699677633 
 

Lasso RMSE [0.27885318 0.25456861 0.25887388 0.26576436 0.21014656] 

average RMSE : 0.25364131894103525 
 

Ridge RMSE [0.26573761 0.24385154 0.24599058 0.25583575 0.21616531] 

average RMSE : 0.2455161592022304 
 

RandomForestRegressor RMSE [0.27209715 0.25170576 0.26211702 0.26120663 0.21941673] 

average RMSE : 0.253308657930002 
 



In [28]:
predictors=['O3','PM10_log','PM25_log','NO2_log','T2M']

In [29]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.26538614 0.24372114 0.24603913 0.25564884 0.21685364] 

average RMSE : 0.24552977824727984 
 

Lasso RMSE [0.27929634 0.25339599 0.25973473 0.266214   0.20971769] 

average RMSE : 0.2536717498662236 
 

Ridge RMSE [0.26538775 0.24372201 0.24604302 0.25564373 0.21683963] 

average RMSE : 0.2455272255973945 
 

RandomForestRegressor RMSE [0.27209715 0.25170576 0.26211702 0.26120663 0.21941673] 

average RMSE : 0.253308657930002 
 



In [30]:
predictors=['O3','PM10_log','PM25_log','NO2_log','T2M_rolling']

In [31]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.25155239 0.22712925 0.23355815 0.23378694 0.19377511] 

average RMSE : 0.22796036576314532 
 

Lasso RMSE [0.27481783 0.24113615 0.2507425  0.25340696 0.19165394] 

average RMSE : 0.24235147415525443 
 

Ridge RMSE [0.25155614 0.22712616 0.23356111 0.23378118 0.19376374] 

average RMSE : 0.2279576652270535 
 

RandomForestRegressor RMSE [0.27247375 0.245499   0.25977973 0.25826535 0.1943188 ] 

average RMSE : 0.2460673252607887 
 



In [32]:
predictors=['O3','PM10_log','PM25_log','NO2_log','T2M_rolling','year','year^2','doy','doy^2']

In [33]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.23254874 0.23921699 0.22731851 0.22662792 0.35888154] 

average RMSE : 0.25691874212490495 
 

Lasso RMSE [0.25560805 0.24021627 0.24247718 0.2659596  0.24291558] 

average RMSE : 0.24943533679256355 
 

Ridge RMSE [0.23454083 0.22509922 0.23005836 0.23010584 0.2357758 ] 

average RMSE : 0.23111601199135615 
 

RandomForestRegressor RMSE [0.26795454 0.24061164 0.25503314 0.26202193 0.19208895] 

average RMSE : 0.2435420386323374 
 



In [34]:
predictors=['T2M_rolling', 'year', 'year^2', 'doy', 'doy^2','E12000001', 'E12000002', 'E12000003', 'E12000004','E12000005', 'E12000006', 'E12000007', 'E12000008']

In [35]:
models = [
    LinearRegression(),
    linear_model.Lasso(alpha=0.1),
    Ridge(alpha=1.0),
    RandomForestRegressor(max_depth=2, random_state=0),
]
CV = 5
for model in models:
    model_name = model.__class__.__name__
    
    accuracies = cross_val_score(model,X[predictors], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

LinearRegression RMSE [0.20415681 0.18456036 0.17517828 0.16492889 0.24821156] 

average RMSE : 0.19540717885674203 
 

Lasso RMSE [0.25597775 0.24258855 0.24892019 0.2694917  0.23816366] 

average RMSE : 0.2510283706219428 
 

Ridge RMSE [0.2017823  0.18332612 0.17372941 0.16563076 0.19561269] 

average RMSE : 0.18401625729827167 
 

RandomForestRegressor RMSE [0.24741372 0.22554667 0.19961574 0.21522269 0.19597469] 

average RMSE : 0.2167547019932517 
 



In [36]:
predictors1=['NO2_log', 'year', 'year^2', 'doy', 'doy^2','PM10_log']
predictors2=['T2M_rolling', 'year', 'year^2', 'doy', 'doy^2','E12000001', 'E12000002', 'E12000003', 'E12000004','E12000005', 'E12000006', 'E12000007', 'E12000008']
predictors3=['E12000001', 'E12000002', 'E12000003', 'E12000004','E12000005', 'E12000006', 'E12000007', 'E12000008']

In [37]:
pred = [
    predictors1,
    predictors2,
    predictors3,
]
for preds in pred:
    #model_name = model.__class__.__name__
    
    accuracies = cross_val_score(LinearRegression(),X[preds], y,scoring=RMSE, cv=CV)
    
    print(model_name,"RMSE",np.sqrt(accuracies),"\n")
    print("average RMSE :", np.sqrt(accuracies).mean(),"\n","\n")

RandomForestRegressor RMSE [0.23201375 0.26048128 0.22666889 0.25644528 0.42542438] 

average RMSE : 0.28020671583860196 
 

RandomForestRegressor RMSE [0.20415681 0.18456036 0.17517828 0.16492889 0.24821156] 

average RMSE : 0.19540717885674203 
 

RandomForestRegressor RMSE [0.28887085 0.26111306 0.24310381 0.22983521 0.22967401] 

average RMSE : 0.25051938878501434 
 



## Final Algo - mixed LR

In [38]:
a = LinearRegression()
b = LinearRegression()
c = LinearRegression()
ab = LinearRegression()

In [39]:

y0 = train_f['mortality_rate']



#modela
a_predictors = ['NO2_log', 'year', 'year^2', 'doy', 'doy^2','PM10_log']
a_X0 = train_f[a_predictors]
a.fit(a_X0, y0)



#modelb
b_predictors = [
        'T2M_rolling', 'year', 'year^2', 'doy', 'doy^2',
        'E12000001', 'E12000002', 'E12000003', 'E12000004',
        'E12000005', 'E12000006', 'E12000007', 'E12000008'
        ]
b_X0 = train_f[b_predictors]
b.fit(b_X0, y0)



#modelc
c_predictors = [
        'E12000001', 'E12000002', 'E12000003', 'E12000004',
        'E12000005', 'E12000006', 'E12000007', 'E12000008'
        ]
c_X0 = train_f[c_predictors]
c.fit(c_X0, y0)


region_effect_reduction = 0.85


# Model AB
# Produces the final solution. Combines A and "reduced B" by using their predictions as features for a linear regression.

ab_X0 = pd.DataFrame({
    'A': a.predict(a_X0),
    'reduced B': b.predict(b_X0) - (c.predict(c_X0) * region_effect_reduction)
    })

ab.fit(ab_X0, y0)

# predict

a_X1 = test_f[a_predictors]
b_X1 = test_f[b_predictors]
c_X1 = test_f[c_predictors]

ab_X1 = pd.DataFrame({'A': a.predict(a_X1),'reduced B': b.predict(b_X1) - (c.predict(c_X1) * region_effect_reduction)})

ab_y1 = ab.predict(ab_X1)



predictions = pd.DataFrame(index=test_f.index)
predictions['mortality_rate'] =  a.predict(a_X1)
predictions.to_csv('C:/Users/deciPh3r/Desktop/Project/files/sub/final_0.0.csv')


# Results
best LB socre for -(region_effect_reduction = 0.85)<br>
private score - 0.28948<br>
public score - 0.28784<br>


benchmark -<br>
private score - 0.29022<br>
public socre - 0.29259<br>